In [78]:
import requests
import datetime
import time
import tweepy
import pandas as pd
import numpy as np
import requests
from decouple import config
import json


def creation_year(year):
    try:
        dt = datetime.datetime.strptime(str(year), '%Y-%m-%d %H:%M:%S')
        return dt.year
    except  Exception as e:
        
        return 0


def convert_string_to_datetime(date):
    print(datetime.datetime.fromtimestamp(date))
    return datetime.datetime.fromtimestamp(date)


def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    if m < n:
        return levenshtein_distance(s2, s1)
    if n == 0:
        return m

    previous_row = range(n+1)
    for i, c1 in enumerate(s1):
        current_row = [i+1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j+1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[n]


def tiktok(tiktokProfile):
    try:

        end_date = datetime.datetime.now()
        start_date = datetime.datetime.now() - datetime.timedelta(days=7)

        num_of_shares_by_u = 0
        num_of_videos_this_week = 0
        monday_to_sunday = [0] * 7
        twelve_am_to_eleven_pm = [0] * 24
        retweeted = 0
        most_recent_post = 0
        tweet_language = ''
        if not tiktokProfile:
            return 'indisponible'
        getUrl = f"https://open-api.tiktok.com/oauth/refresh_token?client_key={config('TIKTOK_KEY')}&grant_type=refresh_token&refresh_token={tiktokProfile['refreshToken']}"
        resMedia = requests.get(getUrl)

        accessToken = json.loads(resMedia.text)['data']['access_token']
        u = requests.get('https://open.tiktokapis.com/v2/user/info/?fields=is_verified,create_time,is_private,follower_count,following_count,likes_count,bio_description,display_name,username,video_count', headers={
            "Authorization": "Bearer " + accessToken,
        })
        payload = "{\"max_count\":20}"
        u = u.json()['data']['user']

        videos = requests.post('https://open.tiktokapis.com/v2/video/list/?fields=create_time,like_count,comment_count,share_count,view_count', headers={
            "Authorization": "Bearer " + accessToken,
            "Content-Type": "application/json"
        }, data=payload)
        videos = videos.json()['data']['videos']
        if len(videos) >= 1:
            most_recent_post = convert_string_to_datetime(videos[0]["create_time"])
            for tweet in videos:
                retweeted += tweet["share_count"]
                tweet_time = convert_string_to_datetime(tweet["create_time"])
                if (tweet_time < end_date and tweet_time > start_date):
                    num_of_videos_this_week += 1
                monday_to_sunday[datetime.datetime.weekday(tweet_time)] += 1
                twelve_am_to_eleven_pm[tweet_time.hour] += 1
        else:
            pass

        u["is_verified"] = int(u["is_verified"])
        uNameScore = 1 - (levenshtein_distance(u["username"], u["display_name"]) / max(
            len(u["username"]), len(u["display_name"])))

        most_recent_post = creation_year(most_recent_post)
        avg_videos_by_hour_of_day = round(
            sum(twelve_am_to_eleven_pm)/len(twelve_am_to_eleven_pm), 3)
        avg_videos_by_day_of_week = round(
            sum(monday_to_sunday)/len(monday_to_sunday), 3)
        u_data = {
            "verified": u["is_verified"],
            "statuses": u["video_count"],
            # "location": u["location"],

            "most_recent_post": most_recent_post,
            "following": u["following_count"],
            "followers": u["follower_count"],
            "favourites": u["likes_count"],

            "videos_this_week": num_of_videos_this_week,

            "retweeted_count": int(retweeted),

            "userNameScore": uNameScore,
            "avg_tweets_by_hour_of_day": avg_videos_by_hour_of_day,
            "avg_tweets_by_day_of_week": avg_videos_by_day_of_week,
            "username": u["username"],
        }
        return u_data
    except Exception as error:
        print('tiktok fetch stats', error)


In [79]:
t = {
    "refreshToken": "rft.e15040f335dfcec11911be52a55dd92eSZyJXmIczOs0DElsGQFmfPxGnwTx!6326"
}
print(tiktok(t))


2023-02-02 11:03:20
2023-02-02 11:03:20
2023-01-19 13:47:18
2022-11-16 14:06:39
2022-11-14 09:53:10
2022-09-16 16:49:22
2022-09-16 15:04:40
2022-08-31 12:04:56
2022-08-12 17:00:37
2022-08-09 16:21:22
2022-07-26 11:17:26
2022-07-14 17:12:40
2022-07-04 17:02:02
2022-06-28 17:25:38
2022-06-23 17:19:58
2022-06-23 13:35:53
2022-06-22 17:54:34
{'verified': 0, 'statuses': 16, 'most_recent_post': 2023, 'following': 4, 'followers': 22, 'favourites': 224, 'videos_this_week': 0, 'retweeted_count': 0, 'userNameScore': 0.5, 'avg_tweets_by_hour_of_day': 0.667, 'avg_tweets_by_day_of_week': 2.286, 'username': 'marwa166ab'}


In [61]:
import requests
import datetime
import time
import tweepy
import pandas as pd
import numpy as np
import requests
from decouple import config
import json


def tiktok(tiktokProfile):
    try:

        if not tiktokProfile:
            return 'indisponible'
        getUrl = f"https://open-api.tiktok.com/oauth/refresh_token?client_key={config('TIKTOK_KEY')}&grant_type=refresh_token&refresh_token={tiktokProfile['refreshToken']}"
        resMedia = requests.get(getUrl)

        accessToken = json.loads(resMedia.text)['data']['access_token']
        u = requests.get('https://open.tiktokapis.com/v2/user/info/?fields=is_verified,created_at,is_private,follower_count,following_count,likes_count,bio_description,display_name,username,video_count', headers={
            "Authorization": "Bearer " + accessToken,
        })
        payload = "{\"max_count\":20}"
        u = u.json()['data']['user']
        videos = requests.post('https://open.tiktokapis.com/v2/video/list/?fields=create_time,like_count,comment_count,share_count,view_count', headers={
            "Authorization": "Bearer " + accessToken,
            "Content-Type": "application/json"
        }, data=payload)
        videos = videos.json()['data']['videos']
        return {"u": u, "videos": videos}
    except Exception as error:
        print('tiktok fetch stats', error)


In [62]:
t = {
    "refreshToken": "rft.e15040f335dfcec11911be52a55dd92eSZyJXmIczOs0DElsGQFmfPxGnwTx!6326"
}
print(tiktok(t))


{'u': {'is_verified': False, 'likes_count': 224, 'video_count': 16, 'bio_description': '', 'display_name': 'marwa', 'follower_count': 22, 'following_count': 4}, 'videos': [{'like_count': 168, 'share_count': 0, 'view_count': 494, 'comment_count': 0, 'create_time': 1675332200}, {'comment_count': 0, 'create_time': 1674132438, 'like_count': 4, 'share_count': 0, 'view_count': 2}, {'like_count': 9, 'share_count': 0, 'view_count': 9, 'comment_count': 0, 'create_time': 1668603999}, {'create_time': 1668415990, 'like_count': 3, 'share_count': 0, 'view_count': 0, 'comment_count': 0}, {'like_count': 2, 'share_count': 0, 'view_count': 0, 'comment_count': 0, 'create_time': 1663343362}, {'comment_count': 0, 'create_time': 1663337080, 'like_count': 0, 'share_count': 0, 'view_count': 0}, {'comment_count': 0, 'create_time': 1661943896, 'like_count': 2, 'share_count': 0, 'view_count': 0}, {'create_time': 1660320037, 'like_count': 4, 'share_count': 0, 'view_count': 1, 'comment_count': 0}, {'comment_count'